In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1️⃣ Dataset paths
# -------------------------------
base_dir = os.path.join(os.getcwd(), "/content/drive/MyDrive/Audio DataSet/alzheimers_dataset")
csv_path = os.path.join(base_dir, "/content/drive/MyDrive/Audio DataSet/alzheimers_dataset/labels.csv")

# -------------------------------
# 2️⃣ Load labels
# -------------------------------
df = pd.read_csv(csv_path)
print(f"Total files: {len(df)}")

# -------------------------------
# 3️⃣ Extract MFCC features
# -------------------------------
def extract_mfcc(file_path, n_mfcc=40):
    y, sr = librosa.load(os.path.join(base_dir, file_path), sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfcc.T, axis=0)  # average across time

print("Extracting MFCC features...")
X = []
y = []

for index, row in df.iterrows():
    file_path = row['filename']
    label = row['label']
    mfcc = extract_mfcc(file_path)
    X.append(mfcc)
    y.append(label)

X = np.array(X)
y = np.array(y)

print(f"Feature shape: {X.shape}, Labels shape: {y.shape}")

# -------------------------------
# 4️⃣ Scale features
# -------------------------------
scaler = StandardScaler()
X = scaler.fit_transform(X)

# -------------------------------
# 5️⃣ Train/Test split
# -------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training samples: {len(X_train)}, Testing samples: {len(X_test)}")

# -------------------------------
# 6️⃣ Build DNN Model
# -------------------------------
model = Sequential([
    Dense(128, activation='relu', input_shape=(X.shape[1],)),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# -------------------------------
# 7️⃣ Train model
# -------------------------------
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=8,
    callbacks=[early_stop]
)

# -------------------------------
# 8️⃣ Evaluate model
# -------------------------------
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc*100:.2f}%")

# -------------------------------
# 9️⃣ Save model
# -------------------------------
model.save("alzheimers_speech_model.h5")
print("✅ Model saved as alzheimers_speech_model.h5")

Total files: 100
Extracting MFCC features...
Feature shape: (100, 40), Labels shape: (100,)
Training samples: 80, Testing samples: 20


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         5,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,569 (53.00 KB)

 Trainable params: 13,569 (53.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - accuracy: 0.5337 - loss: 0.7671 - val_accuracy: 0.6000 - val_loss: 0.6333
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6153 - loss: 0.6333 - val_accuracy: 0.7500 - val_loss: 0.5223
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7837 - loss: 0.5147 - val_accuracy: 1.0000 - val_loss: 0.4269
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9359 - loss: 0.4000 - val_accuracy: 1.0000 - val_loss: 0.3511
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9201 - loss: 0.3462 - val_accuracy: 1.0000 - val_loss: 0.2848
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9808 - loss: 0.2639 - val_accuracy: 1.0000 - val_loss: 0.2199
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9410 - loss: 0.2422 - val_accuracy: 1.0000 - val_loss: 0.1690
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9620 - loss: 0.1746 - val_accuracy: 1.0000 - 

Test Accuracy: 100.00%
✅ Model saved as alzheimers_speech_model.h5


In [8]:
import joblib
joblib.dump(scaler, "scaler.pkl")
print("✅ Scaler saved as scaler.pkl")

✅ Scaler saved as scaler.pkl


In [9]:
# predict.py

import librosa
import numpy as np
import os
import joblib
from tensorflow.keras.models import load_model

# -------------------------------
# Load model & scaler
# -------------------------------
model = load_model("alzheimers_speech_model.h5")
scaler = joblib.load("scaler.pkl")

print("✅ Model and scaler loaded")

# -------------------------------
# MFCC extraction
# -------------------------------
def extract_mfcc(audio_path, n_mfcc=40):
    y, sr = librosa.load(audio_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfcc.T, axis=0)

# -------------------------------
# Prediction
# -------------------------------
def predict_audio(audio_path):
    if not os.path.exists(audio_path):
        print("❌ File not found")
        return

    mfcc = extract_mfcc(audio_path)
    mfcc = mfcc.reshape(1, -1)

    # ✅ USE TRAINING SCALER
    mfcc = scaler.transform(mfcc)

    prob = model.predict(mfcc)[0][0]

    print(f"Raw probability: {prob:.3f}")

    if prob >= 0.5:
        print("🧠 Prediction: Alzheimer’s Disease (AD)")
    else:
        print("🧠 Prediction: Healthy Control (HC)")

# -------------------------------
# Give path
# -------------------------------
audio_path = r"/content/drive/MyDrive/Audio DataSet/alzheimers_dataset/HC/HC_01.wav"
predict_audio(audio_path)

✅ Model and scaler loaded


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 682ms/step
Raw probability: 0.000
🧠 Prediction: Healthy Control (HC)


In [10]:
# predict.py

import librosa
import numpy as np
import os
import joblib
from tensorflow.keras.models import load_model

# -------------------------------
# Load model & scaler
# -------------------------------
model = load_model("alzheimers_speech_model.h5")
scaler = joblib.load("scaler.pkl")

print("✅ Model and scaler loaded")

# -------------------------------
# MFCC extraction
# -------------------------------
def extract_mfcc(audio_path, n_mfcc=40):
    y, sr = librosa.load(audio_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfcc.T, axis=0)

# -------------------------------
# Prediction
# -------------------------------
def predict_audio(audio_path):
    if not os.path.exists(audio_path):
        print("❌ File not found")
        return

    mfcc = extract_mfcc(audio_path)
    mfcc = mfcc.reshape(1, -1)

    # ✅ USE TRAINING SCALER
    mfcc = scaler.transform(mfcc)

    prob = model.predict(mfcc)[0][0]

    print(f"Raw probability: {prob:.3f}")

    if prob >= 0.5:
        print("🧠 Prediction: Alzheimer’s Disease (AD)")
    else:
        print("🧠 Prediction: Healthy Control (HC)")

# -------------------------------
# Give path
# -------------------------------
audio_path = r"/content/drive/MyDrive/Audio DataSet/alzheimers_dataset/AD/AD_01.wav"
predict_audio(audio_path)

✅ Model and scaler loaded


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 639ms/step
Raw probability: 1.000
🧠 Prediction: Alzheimer’s Disease (AD)
